# [[데이콘] 영화 리뷰 감성 분석](https://dacon.io/competitions/official/235995/overview/description)

**참고 사이트**  
[【실습】 Python >> Text Mining -- 감성 분류 분석 (호텔 리뷰 데이터)](https://hyemin-kim.github.io/2020/08/29/E-Python-TextMining-2/)  
[[자연어 처리] konlpy 설치 오류, okt()에러-already loaded in](https://byeon-sg.tistory.com/entry/%EC%9E%90%EC%97%B0%EC%96%B4-%EC%B2%98%EB%A6%AC-konlpy-%EC%84%A4%EC%B9%98-%EC%98%A4%EB%A5%98-okt%EC%97%90%EB%9F%AC-already-loaded-in-another-classloader-SystemErro-1)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## 데이터 셋 불러오기

In [2]:
train_data = pd.read_csv('data/[기업은행 혁신리그 사전강의] 영화 리뷰 감성 분석/train.csv')
train_data

,id,document,label
0,1,아 더빙.. 진짜 짜증나네요 목소리,0
1,2,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,3,너무재밓었다그래서보는것을추천한다,0
3,4,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,5,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,149996,인간이 문제지.. 소는 뭔죄인가..,0
149996,149997,평점이 너무 낮아서...,1
149997,149998,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,149999,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [13]:
# 결측치 확인
train_data.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [32]:
# 결측치 행 확인
train_data[train_data['document'].isnull()]

,id,document,label
25857,25858,NaN,1
55737,55738,NaN,1
110014,110015,NaN,0
126782,126783,NaN,0
140721,140722,NaN,0


In [3]:
# 결측치 행 제거
train_data.dropna(axis=0, inplace=True)

# 결측치 확인
train_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

## 텍스트 데이터 전처리

### 정규 표현식 적용

In [4]:
# 정규 표현식 함수 정의
import re

def apply_regular_expression(text):
    hangle = re.compile('[^ ㄱ-ㅣ 가-힣]') # 한글 추출 규칙: 띄어 쓰기(1개)를 포함한 한글
    result = hangle.sub("", text) # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

### 한글 맞춤법 검사

In [14]:
# !pip install py-hanspell
from hanspell import spell_checker

sent = "한글 맞춤법검사기 재대로작동돼는지테스트"
spelled_sent = spell_checker.check(sent)
checked_sent = spelled_sent.checked

print(checked_sent)

한글 맞춤법검사기 제대로 작동되는지 테스트


### 한국어 형태소 분석

In [5]:
# !pip install konlpy
from konlpy.tag import Okt
from collections import Counter

In [6]:
train_data['document'][0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [7]:
apply_regular_expression(train_data['document'][0])

'아 더빙 진짜 짜증나네요 목소리'

In [8]:
okt = Okt()  # 명사 형태소 추출 함수
nouns = okt.nouns(apply_regular_expression(train_data['document'][0]))
# nouns

In [9]:
# 말뭉치 생성
corpus = "".join(train_data['document'].tolist())
# corpus

In [21]:
# 전체 말뭉치(corpus)에서 명사 형태소 추출
apply_regular_expression(corpus)
nouns = okt.nouns(apply_regular_expression(corpus))
print(nouns)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [22]:
# 빈도 탐색
counter = Counter(nouns)

# 상위 빈도 10개 단어
counter.most_common(10)

[('영화', 49652),
 ('정말', 9452),
 ('진짜', 8285),
 ('점', 7111),
 ('평점', 6307),
 ('연기', 6207),
 ('것', 6207),
 ('최고', 6029),
 ('이', 5399),
 ('스토리', 5313)]

In [59]:
# 한 글자 명사 제거
available_counter = []

for i in range(len(nouns)):
    if len(nouns[i]) > 1:
        available_counter.append(nouns[i])

# 빈도 탐색        
available_counter = Counter(available_counter)

# 한 글자 명사를 제거한 상위 빈도 10개 단어
available_counter.most_common(10)

[('영화', 49652),
 ('정말', 9452),
 ('진짜', 8285),
 ('평점', 6307),
 ('연기', 6207),
 ('최고', 6029),
 ('스토리', 5313),
 ('생각', 5312),
 ('드라마', 5045),
 ('감동', 4868)]

### 불용어 사전 
RANKS NL에 제공해주는 한국어 불용어 사전 활용

In [61]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해']]

In [63]:
add_stopwords = ['구먼', '무재', '밓었', '']

Counter({'더빙': 572,
         '진짜': 8285,
         '목소리': 375,
         '포스터': 555,
         '보고': 4668,
         '초딩': 423,
         '영화': 49652,
         '오버': 141,
         '연기': 6207,
         '무재': 68,
         '밓었': 1,
         '다그': 307,
         '래서': 25,
         '추천': 1177,
         '교도소': 16,
         '이야기': 2161,
         '구먼': 12,
         '재미': 3928,
         '평점': 6307,
         '조정': 39,
         '몬페': 2,
         '스파이더맨': 64,
         '커스틴': 1,
         '던스트': 2,
         '걸음': 20,
         '초등학교': 116,
         '학년': 121,
         '생인': 6,
         '반개': 202,
         '원작': 1261,
         '긴장감': 831,
         '제대로': 790,
         '이응경': 8,
         '길용우': 4,
         '생활': 126,
         '정말': 9452,
         '해도': 438,
         '그것': 444,
         '납치': 49,
         '감금': 11,
         '반복': 150,
         '드라마': 5045,
         '가족': 936,
         '사람': 4834,
         '액션': 2407,
         '왜케': 129,
         '헐리우드': 143,
         '인피니트': 27,
         '볼때': 321,
         '